# Ibis vs Pandas: A Practical Tutorial

# Setup

First, let's install Ibis and a backend. We’ll use DuckDB as it’s easy and lightweight:

In [32]:
!pip install ibis-framework[duckdb] --quiet

## Connecting to a Backend:
Ibis requires you to connect to a backend. Let’s use DuckDB as an in-process database.

In [ ]:
import ibis

# Connect to an in-memory DuckDB instance(database only exists temporarily in memory
#during the session. Once the session ends (like when you restart the Colab environment
# or exit Python), all the data, tables, and changes are lost.)
con = ibis.connect("duckdb://:memory:")

Option 1: Now let's create a database and save it in a csv file temporarily. Unlike Pandas, Ibis doesn’t load CSV directly into a DataFrame—rather, it creates a connection to a table, or you use con.read_csv to register it as a table. Let’s do that:

In [46]:
import pandas as pd
import duckdb

con = duckdb.connect("my_database.duckdb")

# Create a sample people CSV
df_people = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "Diana", "Ethan"],
    "age": [25, 32, 45, 29, 33],
    "city": ["New York", "Chicago", "New York", "Boston", "Chicago"],
    "salary": [70000, 80000, 120000, 90000, 75000]
})
df_people.to_csv("people.csv", index=False)

# Drop the table if it exists, then create a new one from the CSV
con.execute("DROP TABLE IF EXISTS people")
con.execute("CREATE TABLE people AS SELECT * FROM read_csv_auto('people.csv')")

# Now connect Ibis to the DuckDB database
ibis_conn = ibis.duckdb.connect(database="my_database.duckdb")

# Load the 'people' table using Ibis
t = ibis_conn.table("people")

# Check the data
print(t.execute())


      name  age      city  salary
0    Alice   25  New York   70000
1      Bob   32   Chicago   80000
2  Charlie   45  New York  120000
3    Diana   29    Boston   90000
4    Ethan   33   Chicago   75000


option 2: Do not have a duckdb database and want to create one and load existing csv?

In [35]:
import duckdb

# Step 1: Connect to DuckDB (creates 'my_database.duckdb' if it doesn't exist)
con = duckdb.connect('my_database.duckdb')

# Step 2: Load CSV into a DuckDB table
con.execute("""
    CREATE TABLE IF NOT EXISTS people AS
    SELECT * FROM read_csv_auto('people.csv')
""")

# Step 3: Verify that the data was loaded into the DuckDB file
result = con.execute("SELECT * FROM people LIMIT 5").fetchdf()
print(result)




      name  age      city  salary
0    Alice   25  New York   70000
1      Bob   32   Chicago   80000
2  Charlie   45  New York  120000
3    Diana   29    Boston   90000
4    Ethan   33   Chicago   75000


Continuing with option 1, t is now an Ibis table expression, not data loaded in memory. Ibis creates a lazy representation of this table. If you do t in a cell:

In [ ]:
t

DatabaseTable: people
  name   string
  age    int64
  city   string
  salary int64

You’ll see the schema, but Ibis won’t have actually pulled all rows into memory yet.

# Basic Operations:
Selection, Filtering, Grouping, Aggregation

Selecting Columns-

In Pandas, you’d do something like df[['name', 'age']] to select columns. In Ibis:

In [ ]:
#selection in ibis
t[['name', 'age']].execute()

,name,age
0,Alice,25
1,Bob,32
2,Charlie,45
3,Diana,29
4,Ethan,33


Filtering Rows -

In Pandas, df[df['city'] == 'New York'] would filter rows. In Ibis, it’s similar:

In [ ]:
filtered = t[t['city'] == 'New York']
filtered.execute()

,name,age,city,salary
0,Alice,25,New York,70000
1,Charlie,45,New York,120000


Adding New Columns-

In Pandas, df['age_plus_one'] = df['age'] + 1. In Ibis, you use .mutate(): bold text

In [ ]:
with_new_col = t.mutate(age_plus_one=t['age'] + 1)
with_new_col.execute()

,name,age,city,salary,age_plus_one
0,Alice,25,New York,70000,26
1,Bob,32,Chicago,80000,33
2,Charlie,45,New York,120000,46
3,Diana,29,Boston,90000,30
4,Ethan,33,Chicago,75000,34


Grouping and Aggregating-

In Pandas, you might write df.groupby('city')['salary'].mean(). In Ibis, grouping and aggregations are done like:

In [42]:
agg = t.group_by('city').aggregate(
    avg_salary=t['salary'].mean(),
    count_names=t['name'].count()
)
agg.execute()

,city,avg_salary,count_names
0,New York,95000.0,2
1,Boston,90000.0,1
2,Chicago,77500.0,2


This will push down the aggregation to DuckDB, which computes it efficiently, and returns the result as a Pandas DataFrame.

Sorting -

In Pandas: df.sort_values('salary')

In [43]:
t.order_by('salary').execute()

,name,age,city,salary
0,Alice,25,New York,70000
1,Ethan,33,Chicago,75000
2,Bob,32,Chicago,80000
3,Diana,29,Boston,90000
4,Charlie,45,New York,120000


Joins -

We will create a second table called departments. This will be used to demonstrate how to perform joins in Ibis.

In [56]:
# Create 'departments' table
df_departments = pd.DataFrame({
    "city": ["New York", "Chicago", "Boston", "San Francisco"],
    "avg_rent": [3000, 1800, 2500, 4000],
    "region": ["East", "Midwest", "East", "West"]
})
df_departments.to_csv("departments.csv", index=False)
con.execute("DROP TABLE IF EXISTS departments")
con.execute("CREATE TABLE departments AS SELECT * FROM read_csv_auto('departments.csv')")

# Load the tables into Ibis
t = ibis_conn.table("people")
t2 = ibis_conn.table("departments")

# Perform a join on the 'city' column
joined = t.join(t2, t.city == t2.city)

# Select specific columns from the joined result
selected = joined[['name', 'age', 'city', 'salary', 'avg_rent', 'region']]

# Execute the query and print the results
result = selected.execute()
print(result)

      name  age      city  salary  avg_rent   region
0    Alice   25  New York   70000      3000     East
1      Bob   32   Chicago   80000      1800  Midwest
2  Charlie   45  New York  120000      3000     East
3    Diana   29    Boston   90000      2500     East
4    Ethan   33   Chicago   75000      1800  Midwest


Lazy execution of the basic operations

In [57]:
# Combine everything into one lazy query
query = (
    t[['name', 'age', 'salary', 'city']]  # 1 Select specific columns
    .filter(t['city'] == 'New York')  # 2️ Filter for city = 'New York'
    .mutate(age_plus_one=t['age'] + 1)  # 3️ Add age_plus_one column
    .group_by('city')  # 4️⃣ Group by city
    .aggregate(
        avg_salary=t['salary'].mean(),  # Average salary
        count_names=t['name'].count()  # Count of names
    )
    .join(t2, t['city'] == t2['city'])  # 5️ Join with departments on city
    .order_by(ibis.desc('avg_salary'))  # 6️ Order by avg_salary descending
)

# Execute the full lazy query
result = query.execute()

# Display the result
print(result)


       city  avg_salary  count_names  avg_rent region
0  New York     95000.0            2      3000   East


Same query code can be used by different databases duckdb and spark in this case. You will get the same output.

In [60]:
!pip install 'ibis-framework' 'pyspark' --quiet

In [67]:
import ibis
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Ibis Spark Example") \
    .master("local[*]") \
    .getOrCreate()

# Create a DataFrame
df = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "Diana", "Ethan"],
    "age": [25, 32, 45, 29, 33],
    "city": ["New York", "Chicago", "New York", "Boston", "Chicago"],
    "salary": [70000, 80000, 120000, 90000, 75000]
})

# Perform Ibis operations on it
query = (
    t[['name', 'age', 'salary', 'city']]  # 1️⃣ Select specific columns
    .filter(t['city'] == 'New York')  # 2️⃣ Filter for city = 'New York'
    .mutate(age_plus_one=t['age'] + 1)  # 3️⃣ Add age_plus_one column
    .group_by('city')  # 4️⃣ Group by city
    .aggregate(
        avg_salary=t['salary'].mean(),  # Average salary
        count_names=t['name'].count()  # Count of names
    )
    .join(t2, t['city'] == t2['city'])  # 5️⃣ Join with departments on city
    .order_by(ibis.desc('avg_salary'))  # 6️⃣ Order by avg_salary descending
)

# Execute the query
result = query.execute()

# Show the result
print(result)


       city  avg_salary  count_names  avg_rent region
0  New York     95000.0            2      3000   East
